# Inventory Analysis Case Study

Inventory data analysis involves examining and interpreting
data related to a company's inventory to gain insights, make
informed decisions, and optimize inventory management
processes.

Python with libraries like Pandas, Matplotlib, and Seaborn is
commonly used for this type of analysis.

# Inventory Optimization and Sustainability Analysis

Objectives:

    Minimize waste by optimizing inventory levels.
    Identify the most efficient purchasing strategies based on sales, procurement, and inventory data.
    Evaluate product sales performance to formulate a sustainable inventory management approach.

1. Data Preprocessing:

    Consolidate all CSV files into a unified master dataset.
    Screen for any missing or erroneous entries.
    Standardize date formats for consistent time series analysis.

2. Inventory Analysis:

    Assess inventory status at the year's start and end using BegInvFINAL12312016.csv and EndInvFINAL12312016.csv.
    Pinpoint products with the highest and lowest inventory presence.

3. Sales Analysis:

    Examine SalesFINAL12312016.csv to identify bestsellers and products with sluggish sales.
    Analyze sales trends over time, considering variables such as sales quantity, sales price, and date.

4. Purchasing Analysis:

    Evaluate procurement activities using PurchasesFINAL12312016.csv and InvoicePurchases12312016.csv.
    Investigate purchase volumes from different suppliers, procurement costs, and supply chain processes.

5. Optimal Stock Level Calculation:

    Determine the optimal stock level for each product by leveraging sales, procurement, and inventory data.
    Propose stock levels tailored to the sales velocity of products and supply lead times.

Conclusion:
The insights derived from these analyses will provide recommendations for managing inventory in a more efficient and sustainable manner, aiming to cut costs and prevent overstocking and waste.


In [1]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Loading datasets
purchase_prices = pd.read_csv(r"C:\Users\HP\Desktop\Intern\Alfido tech\task_3\PurchasesFINAL12312016.csv")
beg_inv = pd.read_csv(r'C:\Users\HP\Desktop\Intern\Alfido tech\task_3\BegInvFINAL12312016.csv')
end_inv = pd.read_csv(r'C:\Users\HP\Desktop\Intern\Alfido tech\task_3\EndInvFINAL12312016.csv')
invoice_purchases = pd.read_csv(r'C:\Users\HP\Desktop\Intern\Alfido tech\task_3\InvoicePurchases12312016.csv')
purchases = pd.read_csv(r"C:\Users\HP\Desktop\Intern\Alfido tech\task_3\2017PurchasePricesDec.csv")
sales = pd.read_csv(r"C:\Users\HP\Desktop\Intern\Alfido tech\task_3\SalesFINAL12312016.csv")

In [3]:
print("purchase_prices Columns:")
print(purchase_prices.columns.tolist())

print("\nbeg_inv Columns:")
print(beg_inv.columns.tolist())

print("\nend_inv Columns:")
print(end_inv.columns.tolist())

print("\ninvoice_purchases Columns:")
print(invoice_purchases.columns.tolist())

print("\npurchases Columns:")
print(purchases.columns.tolist())

print("\nsales Columns:")
print(sales.columns.tolist())


purchase_prices Columns:
['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'VendorNumber', 'VendorName', 'PONumber', 'PODate', 'ReceivingDate', 'InvoiceDate', 'PayDate', 'PurchasePrice', 'Quantity', 'Dollars', 'Classification']

beg_inv Columns:
['InventoryId', 'Store', 'City', 'Brand', 'Description', 'Size', 'onHand', 'Price', 'startDate']

end_inv Columns:
['InventoryId', 'Store', 'City', 'Brand', 'Description', 'Size', 'onHand', 'Price', 'endDate']

invoice_purchases Columns:
['VendorNumber', 'VendorName', 'InvoiceDate', 'PONumber', 'PODate', 'PayDate', 'Quantity', 'Dollars', 'Freight', 'Approval']

purchases Columns:
['Brand', 'Description', 'Price', 'Size', 'Volume', 'Classification', 'PurchasePrice', 'VendorNumber', 'VendorName']

sales Columns:
['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity', 'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification', 'ExciseTax', 'VendorNo', 'VendorName']


In [4]:
# Checking for missing data in each dataset
datasets = [purchase_prices, beg_inv, end_inv, invoice_purchases, purchases, sales]
dataset_names = ["purchase_prices", "beg_inv", "end_inv", "invoice_purchases", "purchases", "sales"]

for name, data in zip(dataset_names, datasets):
    missing_values = data.isnull().sum()
    non_zero_missing_values = missing_values[missing_values > 0]
    
    if not non_zero_missing_values.empty:
        print(f"\nMissing values in {name}:")
        print(non_zero_missing_values)


Missing values in purchase_prices:
Size    3
dtype: int64

Missing values in end_inv:
City    1284
dtype: int64

Missing values in purchases:
Description    1
Size           1
Volume         1
dtype: int64


Handling missing values is a pivotal step in our data analysis journey. Different data segments might demand distinct strategies. Let's collaboratively address each dataset and decide on the most appropriate course of action:

purchase_prices:

    Description, Size, and Volume: There's a limited number of missing entries here. We have a few options: removing these specific rows, or, for a more informed approach, substituting these gaps with the most commonly observed value (mode). We'll go with removing these rows

end_inv:

    City: An immediate solution could be to replace the missing 'City' entries with the most frequently occurring city (mode). However, if our 'Store' column has location insights, we might consider using it for a more informed fill. If neither works, marking them as 'Unknown' might be our best bet. We'll try our chances with using 'Store' column.

invoice_purchases:

    Approval: If we interpret 'Approval' as a binary indicator (e.g., Approved/Not Approved), the absences might hint that the status of these invoices hasn't been decided. We can categorize them as 'Pending' or 'Unknown' for clarity.

purchases:

    Size: Mirroring our strategy with 'purchase_prices', we can discard these few rows.



In [5]:
# Handling missing values for purchase_prices dataset
cols_to_check = ['Description', 'Size', 'Volume']
for col in cols_to_check:
    purchase_prices = purchase_prices[purchase_prices[col].notna()]

# Handling missing values for end_inv dataset
if end_inv['Store'].nunique() == end_inv['City'].nunique():
    city_store_mapping = end_inv[['Store', 'City']].drop_duplicates().set_index('Store').to_dict()['City']
    end_inv['City'] = end_inv['City'].fillna(end_inv['Store'].map(city_store_mapping))
else:
    end_inv['City'].fillna('Unknown', inplace=True)

# Handling missing values for invoice_purchases dataset
invoice_purchases['Approval'].fillna('Pending', inplace=True)

# Handling missing values for purchases dataset
purchases = purchases[purchases['Size'].notna()]

datasets = [purchase_prices, beg_inv, end_inv, invoice_purchases, purchases, sales]
dataset_names = ["purchase_prices", "beg_inv", "end_inv", "invoice_purchases", "purchases", "sales"]

for name, data in zip(dataset_names, datasets):
    missing_values = data.isnull().sum()
    non_zero_missing_values = missing_values[missing_values > 0]
    
    if not non_zero_missing_values.empty:
        print(f"\nMissing values in {name}:")
        print(non_zero_missing_values)
    else:
        print(f"\nNo missing values in {name}.")

KeyError: 'Volume'

# 2. Inventory Analysis:

In [ ]:

# Grouping by Brand and Description and summarize inventory for beginning of the year
beg_summary = beg_inv.groupby(['Brand', 'Description'])['onHand'].sum().sort_values(ascending=False)

# Grouping by Brand and Description and summarize inventory for end of the year
end_summary = end_inv.groupby(['Brand', 'Description'])['onHand'].sum().sort_values(ascending=False)

# Identifying top 5 products at the beginning and end of the year
top_5_beg = beg_summary.head(5)
top_5_end = end_summary.head(5)

# Identifying bottom 5 products at the beginning and end of the year
bottom_5_beg = beg_summary.tail(5)
bottom_5_end = end_summary.tail(5)

print("Top 5 products at the beginning of the year:\n", top_5_beg)
print("\nTop 5 products at the end of the year:\n", top_5_end)
print("\nBottom 5 products at the beginning of the year:\n", bottom_5_beg)
print("\nBottom 5 products at the end of the year:\n", bottom_5_end)


Insights:

Top Products:

    "Ketel One Vodka" emerged as the top product by the end of the year, even though it was in the 4th position at the beginning. This could imply an increased demand or higher restocking levels for this product during the year.
    "Capt Morgan Spiced Rum", initially the highest in inventory at the start of the year, narrowly missed the top position by a single unit by the year's end.
    "Smirnoff 80 Proof" seems to have undergone a brand update or rebranding, as the brand number changed from 3876 to 8111 during the year. Despite this, its popularity remained consistent.
    "Absolut 80 Proof" held steady, only dropping one rank from the beginning to the end of the year.
    A new entrant, "Jack Daniels No 7 Black", made its way into the top 5 by the end of the year, replacing "Maurice's Mentholated Mint" from the beginning of the year's list.

Bottom Products:

    It's alarming to see products with zero inventory both at the beginning and the end of the year. It's possible that these items either never had stock during the entire year, or they had stock that was completely sold out and never replenished.
    There isn't a significant overlap between the products in the bottom 5 of both lists, indicating that inventory for low-stock items could be fluctuating throughout the year.
    The presence of wines and niche products among the bottom lists suggests that these might be specialty items with selective demand or limited supply.

Inventory Management Observations:

    The overall high consistency among the top products suggests steady demand and effective inventory replenishment strategies for popular items.
    However, the consistent zero counts at both year start and end for certain products indicate potential issues in inventory management or procurement. It may be worthwhile to review the demand for these products and determine if it makes business sense to continue carrying them or if there might be supply chain issues preventing their restocking.


# 3. Sales Analysis

In [ ]:
# Finding the best-selling products
best_selling_products = sales.groupby(['Brand', 'Description']).agg({'SalesQuantity': 'sum'}).sort_values(by='SalesQuantity', ascending=False).head(10)
print(f"Best selling ten products:\n{best_selling_products}\n")

# Finding the slow-moving products
slow_moving_products = sales.groupby(['Brand', 'Description']).agg({'SalesQuantity': 'sum'}).sort_values(by='SalesQuantity', ascending=True).head(10)
print(f"Slow-moving ten products:\n{slow_moving_products}")

In [ ]:
sales['SalesDate'] = pd.to_datetime(sales['SalesDate'])
sales_quantity_trend = sales.groupby('SalesDate').agg({'SalesQuantity': 'sum'})
sales_quantity_trend.plot(figsize=(12, 6), title='Sales Quantity Over Time')

In [ ]:
avg_price_trend = sales.groupby('SalesDate').agg({'SalesPrice': 'mean'})
avg_price_trend.plot(figsize=(12, 6), title='Average Sales Price Over Time')

There's a cyclical nature to the sales quantities, which might hint at weekly patterns or trends.
A significant peak is noticed around January 25th; this could be attributed to specific events, promotions, or even seasonal demands. However, as we transition into February, there's a noticeable decline in sales, which then appears to stabilize as the month progresses.
Spirits like "Smirnoff 80 Proof" emerge as the top sellers, reflecting a steady demand. On the other hand, several products are moving slowly, indicating limited sales. As these products vary in type and brand, a strategic review might be necessary to decide on their continued stocking.

# 4. Purchasing Analysis

In [ ]:
vendor_purchase_volume = invoice_purchases.groupby('VendorName').agg({'Quantity': 'sum'}).sort_values(by='Quantity', ascending=False)
print("Top 10 Vendors by Purchase Volume:\n", vendor_purchase_volume.head(10))

In [ ]:
vendor_purchase_cost = purchases.groupby('VendorName').agg({'PurchasePrice': 'sum'}).sort_values(by='PurchasePrice', ascending=False)
print("Top 10 Vendors by Purchase Cost:\n", vendor_purchase_cost.head(10))

Top Vendors by Purchase Cost:

    DIAGEO NORTH AMERICA INC stands out as the top vendor with the highest purchase cost, amounting to 3,919,293.52 US dollars.
    The following two vendors, 'JIM BEAM BRANDS COMPANY' and 'PERNOD RICARD USA', have notable purchase costs of 2,445,075.37 and 2,002,210.70, respectively.
    It's interesting to see that the top 10 vendors have a significant difference in their purchase costs, with 'DIAGEO NORTH AMERICA INC' nearly leading by a margin of 1.5 million usd from the vendor in the second position.

In [ ]:
# Converting PODate and ReceivingDate columns to datetime
purchases['PoDate'] = pd.to_datetime(purchases['PoDate'], errors='coerce')
purchases['ReceivingDate'] = pd.to_datetime(purchases['ReceivingDate'], errors='coerce')

# Checking if there are any null values after conversion
if purchases['PoDate'].isnull().any() or purchases['ReceivingDate'].isnull().any():
    print("There are invalid date entries in the dataset. Please review the data.")
else:
    purchases['SupplyDuration'] = (purchases['ReceivingDate'] - purchases['PoDate']).dt.days
    average_supply_duration = purchases['SupplyDuration'].mean()
    print("Average Supply Duration (in days):", average_supply_duration)